In [1]:
!pip install mediapipe

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import mediapipe as mp
import os, cv2, json, pickle, time

2025-12-13 00:48:34.321893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765586914.370744     148 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765586914.385439     148 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# From the Dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yousefdotpy/karsl-502")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/karsl-502


In [4]:
# Our cleaned dataset
df = pd.read_excel('/kaggle/input/karsl-502-labels/KARSL-502_Labels.xlsx')
df

,ClassIndex,SignFolder,SignID,Sign-Arabic,Sign-English
0,0,71,71,هيكل عظمي,Skeleton
1,1,72,72,جمجة,skull
2,2,73,73,عمود فقري,Backbone
3,3,74,74,قفص صدري,Chest
4,4,75,75,جهاز تنفسي,Respiratory device
...,...,...,...,...,...
126,126,468,468,مترجم لغة الإشارة,Sign language translator
127,127,496,496,صيدلي,pharmacist
128,128,497,497,طبيب,doctor
129,129,498,498,ممرضة,nurse


In [5]:
df['SignFolder'] = df['SignFolder'].astype(str).str.zfill(4)
#df.set_index('ClassIndex', inplace = True) #this caused error when creating folder_to_class
df

,ClassIndex,SignFolder,SignID,Sign-Arabic,Sign-English
0,0,0071,71,هيكل عظمي,Skeleton
1,1,0072,72,جمجة,skull
2,2,0073,73,عمود فقري,Backbone
3,3,0074,74,قفص صدري,Chest
4,4,0075,75,جهاز تنفسي,Respiratory device
...,...,...,...,...,...
126,126,0468,468,مترجم لغة الإشارة,Sign language translator
127,127,0496,496,صيدلي,pharmacist
128,128,0497,497,طبيب,doctor
129,129,0498,498,ممرضة,nurse


In [6]:
folder_to_class = dict(zip(df['SignFolder'], df['ClassIndex']))
folder_to_class

{'0071': 0,
 '0072': 1,
 '0073': 2,
 '0074': 3,
 '0075': 4,
 '0076': 5,
 '0077': 6,
 '0078': 7,
 '0079': 8,
 '0080': 9,
 '0081': 10,
 '0082': 11,
 '0083': 12,
 '0084': 13,
 '0085': 14,
 '0086': 15,
 '0087': 16,
 '0088': 17,
 '0089': 18,
 '0090': 19,
 '0091': 20,
 '0092': 21,
 '0093': 22,
 '0094': 23,
 '0095': 24,
 '0096': 25,
 '0097': 26,
 '0098': 27,
 '0099': 28,
 '0100': 29,
 '0101': 30,
 '0102': 31,
 '0103': 32,
 '0104': 33,
 '0105': 34,
 '0106': 35,
 '0107': 36,
 '0108': 37,
 '0109': 38,
 '0110': 39,
 '0111': 40,
 '0112': 41,
 '0113': 42,
 '0114': 43,
 '0115': 44,
 '0116': 45,
 '0117': 46,
 '0118': 47,
 '0119': 48,
 '0120': 49,
 '0121': 50,
 '0122': 51,
 '0123': 52,
 '0124': 53,
 '0125': 54,
 '0126': 55,
 '0127': 56,
 '0128': 57,
 '0129': 58,
 '0130': 59,
 '0131': 60,
 '0132': 61,
 '0133': 62,
 '0134': 63,
 '0135': 64,
 '0136': 65,
 '0137': 66,
 '0138': 67,
 '0139': 68,
 '0140': 69,
 '0141': 70,
 '0142': 71,
 '0143': 72,
 '0144': 73,
 '0145': 74,
 '0146': 75,
 '0147': 76,
 '0148': 

In [7]:
karsl_path = '/kaggle/input/karsl-502'

In [ ]:
KARSL_ROOT = karsl_path
OUT_ROOT = "/kaggle/working/karsl_medical_emergencies_signbart"


TARGET_T = 50
MAX_REPS_PER_SIGN = 10  # for train & val (signers 01/02 train, signer 03 val)
DRY_RUN = False  # True = no writing, just counting/logging

#only re-run train+val from original 'train' , test already done :)
ORIGINAL_SPLIT = ["train"] #even for val original is train
#SIGNERS = ["01", "02", "03"]
SIGNERS = ["03"] #for signer 01 - train #for signer 02 - train #for signer 03 - val

for split_name in ["train", "val", "test"]:
    os.makedirs(os.path.join(OUT_ROOT, split_name), exist_ok = True)

label2id = {fol: int(cla) for fol, cla in folder_to_class.items()}
id2label = {str(int(cla)): fol for fol, cla in folder_to_class.items()}

with open(os.path.join(OUT_ROOT, "label2id.json"), "w") as f:
    json.dump(label2id, f)

with open(os.path.join(OUT_ROOT, "id2label.json"), "w") as f:
    json.dump(id2label, f)

print("label2id.json & id2label.json saved in: ", OUT_ROOT)

label2id.json & id2label.json saved in:  /kaggle/working/karsl_medical_emergencies_signbart


In [28]:
def split_target(original_split, signer):
    #only process original split
    if original_split == "train":
        if signer in ["01", "02"]:
            return "train"
        if signer == "03":
            return "val"
    return None

In [29]:
def fix_length(seq, target_t = 50):
    """
    seq: np.ndarray (T, 75, 2)
    Return exactly (target_t, 75, 2) by truncating or padding with last frame.
    """
    T = seq.shape[0]
    if T == target_t:
        return seq
    if T > target_t:
        return seq[:target_t]
    # pad
    pad_n = target_t - T
    last = seq[-1:]
    pad = np.repeat(last, pad_n, axis = 0)
    return np.concatenate([seq, pad], axis = 0)

In [30]:
def extract_keypoints(image_bgr, pose, hands):
    h, w, _ = image_bgr.shape
    img_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    pose_results = pose.process(img_rgb)
    hands_results = hands.process(img_rgb)

    pose_xy = np.zeros((33, 2), dtype = np.float32)
    if pose_results.pose_landmarks:
        for i, lm in enumerate(pose_results.pose_landmarks.landmark):
            pose_xy[i, 0] = lm.x
            pose_xy[i, 1] = lm.y

    left_hand_xy  = np.zeros((21, 2), dtype = np.float32)
    right_hand_xy = np.zeros((21, 2), dtype = np.float32)

    if hands_results.multi_hand_landmarks and hands_results.multi_handedness:
        for hand_landmarks, handedness in zip(hands_results.multi_hand_landmarks, hands_results.multi_handedness):
            hand_label = handedness.classification[0].label  #left or right
            hand_xy = np.array([[lm.x, lm.y] for lm in hand_landmarks.landmark], dtype=np.float32)
            if hand_label == "Left":
                left_hand_xy = hand_xy
            else:
                right_hand_xy = hand_xy

    return np.concatenate([pose_xy, left_hand_xy, right_hand_xy], axis = 0)  # (75,2)

In [31]:
def extract_repetition(rep_path, pose, hands):
    frame_files = sorted([f for f in os.listdir(rep_path) if f.lower().endswith(".jpg")])
    if len(frame_files) == 0:
        return None, "no_frames"

    seq = []
    for frame_name in frame_files:
        frame_path = os.path.join(rep_path, frame_name)
        img = cv2.imread(frame_path)
        if img is None:
            continue
        kps = extract_keypoints(img, pose, hands)
        seq.append(kps)

    if len(seq) == 0:
        return None, "all_frames_failed"

    seq = np.stack(seq, axis = 0) # (T,75,2)
    seq = fix_length(seq, TARGET_T) # (50,75,2)
    return seq, None

In [32]:
def list_rep_dirs(sign_dir):
    """Return repetition folder names that are actual dirs, sorted."""
    if not os.path.isdir(sign_dir):
        return []
    reps = []
    for name in os.listdir(sign_dir):
        p = os.path.join(sign_dir, name)
        if os.path.isdir(p):
            reps.append(name)
    reps.sort()
    return reps

In [33]:
def existing_pkls(out_sign_dir):
    """Return set of filenames already extracted for this sign folder."""
    if not os.path.isdir(out_sign_dir):
        return set()
    return {f for f in os.listdir(out_sign_dir) if f.lower().endswith(".pkl")}

In [15]:
!cp -r /kaggle/input/pre-val/karsl_medical_emergencies_signbart /kaggle/working/

In [16]:
ls /kaggle/working/karsl_medical_emergencies_signbart

id2label.json  resume_log_trainval_20251213_000548.csv  train/
label2id.json  test/                                    val/


In [17]:
ls /kaggle/working/karsl_medical_emergencies_signbart/train | head

0071/
0072/
0073/
0074/
0075/
0076/
0077/
0078/
0079/
0080/


In [18]:
ls /kaggle/working/karsl_medical_emergencies_signbart/test | head

0071/
0072/
0073/
0074/
0075/
0076/
0077/
0078/
0079/
0080/


In [34]:
ls /kaggle/working/karsl_medical_emergencies_signbart/val | head

In [35]:
for split in ["train","val","test"]:
    print(split, len(list(Path(root, split).rglob("*.pkl"))))


train 2620
val 0
test 2957


In [38]:
rows = []
tot_saved_new = 0
tot_already_done = 0
tot_empty_failed = 0
tot_sign_folders_seen = 0
tot_sign_folders_in_subset = 0

mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

with mp_pose.Pose(static_image_mode = True) as pose, mp_hands.Hands(static_image_mode = True, max_num_hands = 2) as hands:

    for original_split in ORIGINAL_SPLIT:
        for signer in SIGNERS:

            target_split = split_target(original_split, signer)
            if target_split is None:
                continue

            base_dir = os.path.join(KARSL_ROOT, signer, signer, original_split) #goes like 01/01/train
            if not os.path.isdir(base_dir):
                print(f"Missing base_dir: {base_dir}")
                continue

            print(f"\n____ Processing signer {signer} | original {original_split} >> target {target_split} ____")
            sign_folders = sorted(os.listdir(base_dir))

            for sign_folder in tqdm(sign_folders):
                tot_sign_folders_seen += 1

                #keep our 131-sign subset
                if sign_folder not in folder_to_class:
                    continue

                tot_sign_folders_in_subset += 1

                in_sign_dir = os.path.join(base_dir, sign_folder)
                if not os.path.isdir(in_sign_dir):
                    continue

                out_sign_dir = os.path.join(OUT_ROOT, target_split, sign_folder)
                os.makedirs(out_sign_dir, exist_ok=True)

                reps_all = list_rep_dirs(in_sign_dir)
                reps_sel = reps_all[:MAX_REPS_PER_SIGN]  #take first 10 (consistent)

                done_set = existing_pkls(out_sign_dir)

                found_dirs = len(reps_all)
                selected = len(reps_sel)

                already_done = 0
                saved_new = 0
                empty_failed = 0

                for rep_folder in reps_sel:
                    out_name = f"{signer}_{rep_folder}.pkl"
                    out_path = os.path.join(out_sign_dir, out_name)

                    if out_name in done_set or os.path.exists(out_path):
                        already_done += 1
                        continue

                    rep_path = os.path.join(in_sign_dir, rep_folder)
                    if not os.path.isdir(rep_path):
                        empty_failed += 1
                        continue

                    seq, err = extract_repetition(rep_path, pose, hands)
                    if seq is None:
                        empty_failed += 1
                        continue

                    sample = {
                        "keypoints": seq.astype(np.float32), # (50,75,2)
                        "class": int(folder_to_class[sign_folder])  #int label
                    }

                    if not DRY_RUN:
                        with open(out_path, "wb") as f:
                            pickle.dump(sample, f)

                    saved_new += 1
                    tot_saved_new += 1

                tot_already_done += already_done
                tot_empty_failed += empty_failed

                rows.append({
                    "signer": signer,
                    "target_split": target_split,
                    "sign_folder": sign_folder,
                    "class_id": int(folder_to_class[sign_folder]),
                    "found_rep_dirs": found_dirs,
                    "selected_reps": selected,
                    "already_done": already_done,
                    "saved_new": saved_new,
                    "empty_or_failed": empty_failed,
                    "out_dir": out_sign_dir,
                })

print("\nExtraction pass finished :)")
print("Saved NEW:", tot_saved_new)
print("Already existed:", tot_already_done)
print("Empty/failed reps:", tot_empty_failed)
print("Seen sign folders:", tot_sign_folders_seen)
print("Subset sign folders seen:", tot_sign_folders_in_subset)

log_df = pd.DataFrame(rows)
log_path = os.path.join(OUT_ROOT, f"log_val_{time.strftime('%Y%m%d_%H%M%S')}.csv")
log_df.to_csv(log_path, index = False)
print("Log saved:", log_path)


____ Processing signer 03 | original train >> target val ____


  0%|          | 0/502 [00:00<?, ?it/s]W0000 00:00:1765588677.743385     238 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765588677.772858     238 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765588677.790983     233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765588677.858912     233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765588677.910693     234 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
100%|██████████| 502/502 [


Extraction pass finished :)
Saved NEW: 1310
Already existed: 0
Empty/failed reps: 0
Seen sign folders: 502
Subset sign folders seen: 131
Log saved: /kaggle/working/karsl_medical_emergencies_signbart/log_val_20251213_031428.csv


In [39]:
for split in ["train", "val", "test"]:
    n = len(list(Path(root, split).rglob("*.pkl")))
    print(split, ":", n)

train : 2620
val : 1310
test : 2957


In [40]:
!cd /kaggle/working && zip -r karsl_FINAL.zip karsl_medical_emergencies_signbart

  adding: karsl_medical_emergencies_signbart/ (stored 0%)
  adding: karsl_medical_emergencies_signbart/log_val_20251213_011654.csv (stored 0%)
  adding: karsl_medical_emergencies_signbart/log_val_20251213_010744.csv (stored 0%)
  adding: karsl_medical_emergencies_signbart/log_val_20251213_010921.csv (stored 0%)
  adding: karsl_medical_emergencies_signbart/test/ (stored 0%)
  adding: karsl_medical_emergencies_signbart/test/0186/ (stored 0%)
  adding: karsl_medical_emergencies_signbart/test/0186/02_03_02_0186_(16_11_17_01_27_43)_c.pkl (deflated 36%)
  adding: karsl_medical_emergencies_signbart/test/0186/01_03_01_0186_(21_10_17_16_01_47)_c.pkl (deflated 14%)
  adding: karsl_medical_emergencies_signbart/test/0186/01_03_01_0186_(21_10_17_16_02_16)_c.pkl (deflated 15%)
  adding: karsl_medical_emergencies_signbart/test/0186/02_03_02_0186_(16_11_17_01_28_03)_c.pkl (deflated 36%)
  adding: karsl_medical_emergencies_signbart/test/0186/01_03_01_0186_(13_09_17_14_35_43)_c.pkl (deflated 14%)
  addi

In [ ]:
!ls /kaggle/input/all-test-set/karsl_medical_emergencies_signbart/test

In [ ]:
!cp -rn /kaggle/input/all-test-set/karsl_medical_emergencies_signbart/test \
      /kaggle/working/karsl_medical_emergencies_signbart/


In [ ]:
root = "/kaggle/working/karsl_medical_emergencies_signbart"

for split in ["train", "val", "test"]:
    n = len(list(Path(root, split).rglob("*.pkl")))
    print(split, ":", n)

In [ ]:
!ls /kaggle/working/karsl_medical_emergencies_signbart/train/0144 #both signers should be shown ;)

In [ ]:
!cd /kaggle/working && zip -r karsl_train_01_02_test.zip karsl_medical_emergencies_signbart

In [ ]:
!ls -lh /kaggle/working | grep karsl

In [ ]:
!cd /kaggle/working && zip -r karsl_partial_after_signer01.zip karsl_medical_emergencies_signbart
!ls -lh karsl_partial_after_signer01.zip


In [ ]:
!cd /kaggle/working
!unzip -n karsl_partial_after_signer01.zip

In [ ]:
!ls karsl_medical_emergencies_signbart/train/0144 | head